In [1]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.housing_summary
df = pd.DataFrame(list(db.redfin.find().limit(100000)))
# df = pd.DataFrame(list(db.redfin.find()))

In [3]:
sel_cols = ['period_begin',  
'period_end', 
'region_name' ,
'region_type',
'total_homes_sold',
'median_sale_price',
'median_sale_ppsf',
'median_new_listing_price',
'pending_sales',
'total_new_listings',
'price_drops',
'homes_delisted',
'inventory',
'total_active_listings',
'median_days_on_market'
]

In [4]:
df_reduced = df[sel_cols]

In [5]:
# df_reduced["period_begin"].to_datetime()
# 
begin_dates = pd.to_datetime(df_reduced["period_begin"], infer_datetime_format=True)
df_reduced["year"] = begin_dates.dt.strftime("%Y")
df_reduced["month"] = begin_dates.dt.strftime("%m")

df_reduced["year_month"] = begin_dates.dt.strftime("%Y-%m")

states = []
for region in df_reduced["region_name"]:
    reg_list = region.split(",")
    if len(reg_list) == 2:
        reg_state = reg_list[1][1:3]
        states.append(reg_list[1][1:3])
    else:
        states.append("")
df_reduced["state"] = states


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [6]:
df = df_reduced
df.fillna('', inplace=True)
df = df[(df.year == '2020')& (df.state == 'NY')]
df


,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,median_sale_ppsf,median_new_listing_price,pending_sales,total_new_listings,price_drops,homes_delisted,inventory,total_active_listings,median_days_on_market,year,month,year_month,state
23,2020-04-06,2020-04-12,"Chautauqua County, NY",county,21,92000,61.828,34900,8,17,8,2,334,349,101,2020,04,2020-04,NY
339,2020-08-17,2020-08-23,"Allegany County, NY",county,9,92600,62.5411,89900,12,14,3,2,99,113,60,2020,08,2020-08,NY
402,2020-01-20,2020-01-26,"Chautauqua County, NY",county,33,68000,46.3213,84000,10,17,14,2,390,416,59,2020,01,2020-01,NY
445,2020-10-12,2020-10-18,"Sullivan County, NY",county,27,225000,129.268,270000,23,38,18,10,579,617,71,2020,10,2020-10,NY
560,2020-08-24,2020-08-30,"Chautauqua County, NY",county,24,151000,80.6552,129900,33,49,17,3,305,346,48.5,2020,08,2020-08,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99703,2020-06-15,2020-06-21,"Allegany County, NY",county,4,54050,48.9859,102500,9,8,2,2,100,113,96,2020,06,2020-06,NY
99709,2020-04-20,2020-04-26,"Sullivan County, NY",county,11,187500,115.314,194500,8,9,8,2,493,508,100,2020,04,2020-04,NY
99890,2020-07-20,2020-07-26,"Onondaga County, NY",county,78,165000,106.507,183450,124,152,31,11,852,997,27.5,2020,07,2020-07,NY
99939,2020-03-30,2020-04-05,"Ontario County, NY",county,23,172500,91.6667,187900,13,7,5,4,238,256,23,2020,03,2020-03,NY


In [7]:
df[['price_drops','inventory', 'median_days_on_market','median_sale_price' ]]= df[['price_drops','inventory', 'median_days_on_market','median_sale_price' ]].apply(pd.to_numeric)
df.dtypes

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


period_begin                 object
period_end                   object
region_name                  object
region_type                  object
total_homes_sold             object
median_sale_price           float64
median_sale_ppsf             object
median_new_listing_price     object
pending_sales                object
total_new_listings           object
price_drops                 float64
homes_delisted               object
inventory                   float64
total_active_listings        object
median_days_on_market       float64
year                         object
month                        object
year_month                   object
state                        object
dtype: object

In [8]:
df= df.groupby(['month','region_name']).agg({'inventory':'mean',
                         'median_days_on_market':'mean',
                         'median_sale_price':'mean',
                         'price_drops':'mean',
                        })
df

inventory  median_days_on_market  \
month region_name                                                    
01    Albany, NY metro area     2533.000000                   64.5   
      Allegany County, NY        128.333333                  109.5   
      Amsterdam, NY metro area   151.000000                   35.0   
      Batavia, NY metro area      79.000000                   56.5   
      Bronx County, NY          1115.000000                  119.5   
...                                     ...                    ...   
12    Suffolk County, NY        3353.500000                   31.5   
      Sullivan County, NY        448.666667                   74.5   
      Syracuse, NY metro area   1040.000000                   32.0   
      Washington County, NY      189.000000                   49.0   
      Wyoming County, NY          52.000000                   15.5   

                                median_sale_price  price_drops  
month region_name                                               
01    Albany, NY metro area              191000.0   100.000000  
      Allegany County, NY                 61000.0     2.666667  
      Amsterdam, NY metro area            92000.0     4.000000  
      Batavia, NY metro area             144000.0     2.000000  
      Bronx County, NY                   532500.0    29.000000  
...                                           ...          ...  
12    Suffolk County, NY                 485375.0    79.500000  
      Sullivan County, NY                201750.0     6.666667  
      Syracuse, NY metro area            160000.0     9.000000  
      Washington County, NY              174000.0     3.000000  
      Wyoming County, NY                 139975.0     4.000000  

[459 rows x 4 columns]

In [9]:
df.reset_index(inplace = True)
data_dict = df.to_dict("records")

In [10]:
 # Insert dictionary into MongoDB as a document
    collection.insert_many(data_dict)

In [11]:
df = pd.DataFrame(list(db.housing_summary.find()))
df

,_id,month,region_name,inventory,median_days_on_market,median_sale_price,price_drops
0,60960359c550f459e8e988e1,01,"Albany, NY metro area",2533.000000,64.5,191000.0,100.000000
1,60960359c550f459e8e988e2,01,"Allegany County, NY",128.333333,109.5,61000.0,2.666667
2,60960359c550f459e8e988e3,01,"Amsterdam, NY metro area",151.000000,35.0,92000.0,4.000000
3,60960359c550f459e8e988e4,01,"Batavia, NY metro area",79.000000,56.5,144000.0,2.000000
4,60960359c550f459e8e988e5,01,"Bronx County, NY",1115.000000,119.5,532500.0,29.000000
...,...,...,...,...,...,...,...
454,60960359c550f459e8e98aa7,12,"Suffolk County, NY",3353.500000,31.5,485375.0,79.500000
455,60960359c550f459e8e98aa8,12,"Sullivan County, NY",448.666667,74.5,201750.0,6.666667
456,60960359c550f459e8e98aa9,12,"Syracuse, NY metro area",1040.000000,32.0,160000.0,9.000000
457,60960359c550f459e8e98aaa,12,"Washington County, NY",189.000000,49.0,174000.0,3.000000


In [12]:
df['median_days_on_market', 'median_sale_price'] = df['median_days_on_market','median_sale_price'].round(2)

KeyError: ('median_days_on_market', 'median_sale_price')